# Importando Bibliotecas

In [21]:
import gdown
import json
#Import pandas
import pandas as pd

#Import numpy
import numpy as np

#Import sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

#Import regex for text processing
import re

In [22]:
pd.reset_option('display.max_rows')

In [23]:
#Caminho para o Google Drive
file_Prospects  = '1Y5eePChAUU3Lv581ms-OEfvDQdOhFSqZ'
file_Applicants = '1M5mmqRf6XmlZU18u3a1Qv9WLLSAe64pP'
file_Vagas      = '1bG8_reZXL2fkswXvUQoCIvm9UUSPOaGW'

In [24]:
# # Baixar os arquivos do Google Drive
# # Certifique-se de que o gdown está instalado: pip install gdown
# gdown.download(f'https://drive.google.com/uc?export=download&id={file_Prospects}', 'prospects.json', quiet=False)
# gdown.download(f'https://drive.google.com/uc?export=download&id={file_Applicants}', 'applicants.json', quiet=False)
# gdown.download(f'https://drive.google.com/uc?export=download&id={file_Vagas}', 'vagas.json', quiet=False)

In [25]:

# Função para carregar arquivos JSON com tratamento de erros
def load_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"❌ Arquivo não encontrado: {file_path}")
    except json.JSONDecodeError:
        print(f"❌ Erro ao decodificar JSON no arquivo: {file_path}")
    except Exception as e:
        print(f"❌ Erro inesperado no arquivo {file_path}: {e}")

In [26]:
# ✅ Função para remover barras invertidas (\) de strings, listas e dicionários
def remove_backslashes_from_data(data):
    if isinstance(data, str):
        return data.replace("\\", "")  # Corrigido: remover barras invertidas
    elif isinstance(data, dict):
        return {key: remove_backslashes_from_data(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [remove_backslashes_from_data(item) for item in data]
    else:
        return data

In [27]:
# ✅ Função para carregar JSON com segurança
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# ✅ Função genérica para flatten de JSONs
def flatten_json(data, id_name):
    records = []
    for prof_id, profile_info in data.items():
        record = {id_name: prof_id}
        for section_name, section_data in profile_info.items():
            if isinstance(section_data, dict):
                for key, value in section_data.items():
                    record[f"{section_name}__{key}"] = value
            else:
                record[section_name] = section_data
        records.append(record)
    return pd.DataFrame(records)

# ✅ Carregar os arquivos JSON
data_Prospects = load_json('prospects.json')
data_Applicants = load_json('applicants.json')
data_Vagas = load_json('vagas.json')

# ✅ Processar Prospects (estrutura especial por ter lista de candidatos)
prospect_records = []
for prof_id, profile_info in data_Prospects.items():
    titulo = profile_info.get("titulo")
    modalidade = profile_info.get("modalidade")

    for prospect in profile_info.get("prospects", []):
        record = {
            "id_prospect": prof_id,
            "titulo": titulo,
            "modalidade": modalidade,
            "nome_candidato": prospect.get("nome"),
            "codigo_candidato": prospect.get("codigo"),
            "situacao_candidado": prospect.get("situacao_candidado"),
            "data_candidatura": prospect.get("data_candidatura"),
            "ultima_atualizacao": prospect.get("ultima_atualizacao"),
            "comentario": prospect.get("comentario"),
            "recrutador": prospect.get("recrutador")
        }
        prospect_records.append(record)

df_Prospects = pd.DataFrame(prospect_records)

# ✅ Processar Applicants
df_Applicants = flatten_json(data_Applicants, id_name="id_applicant")

# ✅ Processar Vagas
df_Vagas = flatten_json(data_Vagas, id_name="id_vaga")

# ✅ Exemplo de visualização rápida
df_Prospects.head()

,id_prospect,titulo,modalidade,nome_candidato,codigo_candidato,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano
4,4533,2021-2605708-Microfocus Application Life Cycle...,,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira


In [28]:
#Excluindo colunas modalidade e comentario
df_Prospects.drop('modalidade',axis=1,inplace=True)
df_Prospects.drop('comentario',axis=1,inplace=True)

In [29]:
# Contar valores nulos e vazios em cada coluna
empty_counts = (df_Applicants.isnull() | (df_Applicants == '')).sum()

# Indentificar colunas com mais de 40.000 valores vazios
cols_to_drop = empty_counts[empty_counts > 40000].index

# dropar as colunas com mais de 40.000 valores vazios
df_Applicants.drop(columns=cols_to_drop, inplace=True)

df_Applicants.columns

Index(['id_applicant', 'infos_basicas__telefone',
       'infos_basicas__objetivo_profissional', 'infos_basicas__data_criacao',
       'infos_basicas__inserido_por', 'infos_basicas__email',
       'infos_basicas__local', 'infos_basicas__sabendo_de_nos_por',
       'infos_basicas__data_atualizacao', 'infos_basicas__codigo_profissional',
       'infos_basicas__nome', 'informacoes_pessoais__data_aceite',
       'informacoes_pessoais__nome', 'informacoes_pessoais__fonte_indicacao',
       'informacoes_pessoais__email', 'informacoes_pessoais__data_nascimento',
       'informacoes_pessoais__telefone_celular', 'informacoes_pessoais__sexo',
       'informacoes_pessoais__estado_civil', 'informacoes_pessoais__pcd',
       'informacoes_pessoais__endereco',
       'informacoes_profissionais__titulo_profissional',
       'informacoes_profissionais__area_atuacao',
       'informacoes_profissionais__conhecimentos_tecnicos',
       'informacoes_profissionais__remuneracao',
       'formacao_e_idiomas__

In [30]:
# Substituir nulos disfarçados por NaN
df_Applicants['formacao_e_idiomas__nivel_academico'] = df_Applicants['formacao_e_idiomas__nivel_academico'].replace(
    ['', ' ', 'NULL', 'N/A', 'NA', 'None'], pd.NA
)

In [31]:
# Contar valores nulos e vazios em cada coluna
empty_counts = (df_Vagas.isnull() | (df_Vagas == '')).sum()

# Identificar colunas com mais de 13.000 valores vazios
cols_to_drop = empty_counts[empty_counts > 13000].index

# Excluir as colunas com mais de 13.000 valores vazios
df_Vagas.drop(columns=cols_to_drop, inplace=True)

df_Vagas.columns

Index(['id_vaga', 'informacoes_basicas__data_requicisao',
       'informacoes_basicas__limite_esperado_para_contratacao',
       'informacoes_basicas__titulo_vaga', 'informacoes_basicas__vaga_sap',
       'informacoes_basicas__cliente',
       'informacoes_basicas__solicitante_cliente',
       'informacoes_basicas__empresa_divisao',
       'informacoes_basicas__requisitante',
       'informacoes_basicas__analista_responsavel',
       'informacoes_basicas__tipo_contratacao',
       'informacoes_basicas__prazo_contratacao',
       'informacoes_basicas__objetivo_vaga',
       'informacoes_basicas__prioridade_vaga',
       'informacoes_basicas__origem_vaga',
       'informacoes_basicas__superior_imediato', 'perfil_vaga__pais',
       'perfil_vaga__estado', 'perfil_vaga__cidade', 'perfil_vaga__bairro',
       'perfil_vaga__regiao', 'perfil_vaga__local_trabalho',
       'perfil_vaga__vaga_especifica_para_pcd', 'perfil_vaga__faixa_etaria',
       'perfil_vaga__nivel profissional', 'perfil_vag

In [32]:
# Merge: Applicants + Prospects
df = df_Prospects.merge(df_Applicants, left_on='codigo_candidato', right_on='id_applicant', how='left')

# Merge: Com as Vagas
df = df.merge(df_Vagas, left_on='id_prospect', right_on='id_vaga', how='left')

In [33]:
# # Configurar para mostrar todas as colunas
# pd.set_option('display.max_columns', None)
# # Configurar para mostrar todas as linhas
# pd.set_option('display.max_colwidth', None)

In [34]:
import nltk
nltk.download('stopwords')
nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ander\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

In [35]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [36]:
# ===================================================
# 🚀 Função que gera o regex dos termos da vaga
# ===================================================
def gerar_regex(texto):
    if pd.isnull(texto):
        return ''
    
    # 🔧 Pré-processamento
    texto = texto.lower()
    texto = re.sub(r'[\n\r\t]', ' ', texto)
    texto = re.sub(r'[\.\,\;\:\(\)\[\]\{\}\!\?]', ' ', texto)
    
    # 🪪 Extrai palavras
    palavras = re.findall(r'\b[a-zA-ZÀ-ÿ0-9\-\._\+/]{2,}\b', texto)
    
    # 🎯 Filtra palavras relevantes
    termos = [
        palavra for palavra in palavras
        if palavra not in stopwords and not palavra.isnumeric() and len(palavra) >= 4
    ]
    
    termos_unicos = sorted(set(termos))
    
    # 🔥 Gera regex no formato (termo1|termo2|...)
    if termos_unicos:
        regex = r'(' + '|'.join(re.escape(termo) for termo in termos_unicos) + r')'
    else:
        regex = ''
    
    return regex

# ===================================================
# 🚀 Aplica a função para gerar regex para cada linha
# ===================================================
df['regex_termos_vaga'] = df['perfil_vaga__competencia_tecnicas_e_comportamentais'].apply(gerar_regex)

# ===================================================
# 🔍 Verifica se os termos aparecem na coluna 'cv_pt'
# ===================================================
def buscar_termos(cv, regex):
    if pd.isnull(cv) or regex == '':
        return []
    return re.findall(regex, cv.lower())

# 🔎 Encontra os termos no CV com base no regex daquela linha — sem duplicados
df['termos_encontrados'] = df.apply(
    lambda x: list(set(buscar_termos(x['cv_pt'], x['regex_termos_vaga']))), axis=1
)


# ✅ Cria uma flag se encontrou algum termo
df['possui_termo'] = df['termos_encontrados'].apply(
    lambda x: 'Sim' if len(x) > 0 else 'Não'
)

# 🔢 Conta quantos termos aparecem por linha
df['qtd_termos'] = df['termos_encontrados'].apply(len)

In [37]:
# ======================= 📥 SELECIONAR COLUNAS =======================
colunas = [
    "id_prospect", 
    "titulo", 
    "nome_candidato", 
    "codigo_candidato", 
    "situacao_candidado",
    "infos_basicas__objetivo_profissional",
    "informacoes_profissionais__area_atuacao",
    "formacao_e_idiomas__nivel_academico",
    "formacao_e_idiomas__nivel_ingles",
    "formacao_e_idiomas__nivel_espanhol",
    "cv_pt", 
    "perfil_vaga__competencia_tecnicas_e_comportamentais",
    "perfil_vaga__nivel profissional",
    "perfil_vaga__nivel_academico", 
    "regex_termos_vaga", 
    "termos_encontrados", 
    "possui_termo", 
    "qtd_termos"
]

df = df[colunas].fillna('')

# ======================= 🏷️ TARGET =======================
status_positivo = [
    'Encaminhado ao Requisitante',
    'Contratado pela Decision',
    'Contratado como Hunting',
    'Aprovado',
    'Entrevista Técnica',
    'Entrevista com Cliente',
    'Encaminhar Proposta',
    'Proposta Aceita'
]

status_negativo = [
    'Não Aprovado pelo RH',
    'Não Aprovado pelo Cliente',
    'Não Aprovado pelo Requisitante',
    'Desistiu',
    'Desistiu da Contratação',
    'Sem interesse nesta vaga',
    'Recusado'
]

df = df[df['situacao_candidado'].isin(status_positivo + status_negativo)]
df['target'] = df['situacao_candidado'].apply(lambda x: 1 if x in status_positivo else 0)

# ======================= 🧹 FILTRO DE QUALIDADE =======================
df['termos_encontrados'] = df['termos_encontrados'].apply(
    lambda x: eval(x) if isinstance(x, str) else x
)
df = df[~((df['target'] == 1) & (df['termos_encontrados'].apply(lambda x: len(x) == 0)))]

# ======================= 🔠 SIMILARIDADE TF-IDF =======================
vectorizer = TfidfVectorizer()
corpus = df['perfil_vaga__competencia_tecnicas_e_comportamentais'].astype(str).tolist() + \
         df['cv_pt'].astype(str).tolist()

tfidf_matrix = vectorizer.fit_transform(corpus)
tfidf_vaga = tfidf_matrix[:len(df)]
tfidf_cv = tfidf_matrix[len(df):]

df['similaridade_cv_vaga'] = [
    cosine_similarity(tfidf_cv[i], tfidf_vaga[i])[0][0] for i in range(len(df))
]
df['similaridade_cv_vaga'] = (df['similaridade_cv_vaga'] + 1) / 2  # Normalizar para 0-1

# ======================= 📊 DENSIDADE DE TERMOS =======================
def calcular_match_percent(row):
    try:
        if pd.isna(row['regex_termos_vaga']) or row['regex_termos_vaga'] == '':
            return 0
        
        # Verifica se já é uma lista (evitando eval)
        if isinstance(row['regex_termos_vaga'], list):
            termos_vaga = row['regex_termos_vaga']
        else:
            # Remove caracteres problemáticos antes de fazer eval
            termos_str = row['regex_termos_vaga'].strip()
            if termos_str.startswith('(') and termos_str.endswith(')'):
                termos_str = termos_str[1:-1]
            termos_vaga = termos_str.split('|')
        
        total_termos = len(termos_vaga)
        return row['qtd_termos'] / total_termos if total_termos > 0 else 0
    except:
        return 0

df['match_percent'] = df.apply(calcular_match_percent, axis=1)

# ======================= 🎓 FEATURE - ADERÊNCIA ACADÊMICA =======================
def mapear_nivel_academico(nivel):
    if pd.isna(nivel):
        return 0
    nivel = nivel.lower()
    if "doutorado" in nivel: return 5
    if "mestrado" in nivel: return 4
    if "pós graduação" in nivel or "pós-graduação" in nivel: return 3
    if "ensino superior" in nivel: return 2
    if "ensino técnico" in nivel: return 1.5
    if "ensino médio" in nivel: return 1
    if "ensino fundamental" in nivel: return 0.5
    return 0

df['nivel_academico_candidato'] = df['formacao_e_idiomas__nivel_academico'].apply(mapear_nivel_academico)
df['nivel_academico_vaga'] = df['perfil_vaga__nivel_academico'].apply(mapear_nivel_academico)

df['aderencia_academica'] = df.apply(
    lambda x: 1 if x['nivel_academico_candidato'] >= x['nivel_academico_vaga'] else 0.5 if x['nivel_academico_candidato'] >= x['nivel_academico_vaga'] - 1 else 0, 
    axis=1
)

# ======================= 🌐 FEATURE - IDIOMAS =======================
def mapear_nivel_idioma(nivel):
    if pd.isna(nivel):
        return 0
    nivel = nivel.lower()
    if nivel == 'nenhum': return 0
    if nivel == 'básico': return 1
    if nivel == 'intermediário': return 2
    if nivel == 'avançado': return 3
    if nivel == 'fluente': return 4
    return 0

nivel_ingles_vaga = 2  # Default intermediário
nivel_espanhol_vaga = 1  # Default básico

df['nivel_ingles'] = df['formacao_e_idiomas__nivel_ingles'].apply(mapear_nivel_idioma)
df['aderencia_ingles'] = df['nivel_ingles'].apply(
    lambda x: 1 if x >= nivel_ingles_vaga else 0.5 if x >= nivel_ingles_vaga - 1 else 0
)

df['nivel_espanhol'] = df['formacao_e_idiomas__nivel_espanhol'].apply(mapear_nivel_idioma)
df['aderencia_espanhol'] = df['nivel_espanhol'].apply(
    lambda x: 1 if x >= nivel_espanhol_vaga else 0.5 if x >= nivel_espanhol_vaga - 1 else 0
)

# ======================= 💼 FEATURE - NÍVEL PROFISSIONAL DA VAGA =======================
mapa_nivel_profissional = {
    'Aprendiz': 1, 'Trainee': 2, 'Auxiliar': 3, 'Assistente': 4,
    'Técnico de Nível Médio': 5, 'Júnior': 5.5, 'Analista': 6,
    'Pleno': 7, 'Supervisor': 7, 'Líder': 7.5, 'Sênior': 8,
    'Especialista': 9, 'Coordenador': 9, 'Gerente': 10
}

df['nivel_profissional_vaga'] = df['perfil_vaga__nivel profissional'].map(mapa_nivel_profissional).fillna(0)
df['nivel_profissional_norm'] = df['nivel_profissional_vaga'] / 10

# ======================= 🎯 FEATURES FINAIS =======================
features = [
    'match_percent',
    'similaridade_cv_vaga',
    'qtd_termos',
    'aderencia_academica',
    'aderencia_ingles',
    'aderencia_espanhol',
    'nivel_profissional_norm'
]

X = df[features]
y = df['target']

# ======================= 🔧 NORMALIZAÇÃO =======================
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# ======================= ⚖️ BALANCEAMENTO =======================
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_scaled, y)

# ======================= 🤖 MODELO FINAL =======================
modelo_final = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    class_weight='balanced',
    random_state=42
)
modelo_final.fit(X_res, y_res)

# Avaliação do modelo
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42
)
y_pred = modelo_final.predict(X_test)
print(classification_report(y_test, y_pred))

# ======================= 💾 SALVAR MODELO =======================
import joblib
joblib.dump(modelo_final, 'modelo_rf_final.pkl')
joblib.dump(scaler, 'scaler_final.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

# ======================= 🔥 SCORE FINAL =======================
df['prob_apto'] = modelo_final.predict_proba(X_scaled)[:, 1]
df['score_final'] = (df['prob_apto'] * 0.3) + (df['match_percent'] * 0.4) + (df['similaridade_cv_vaga'] * 0.2) + (df['aderencia_academica'] * 0.1)

              precision    recall  f1-score   support

           0       0.91      0.45      0.60      2803
           1       0.63      0.96      0.76      2827

    accuracy                           0.70      5630
   macro avg       0.77      0.70      0.68      5630
weighted avg       0.77      0.70      0.68      5630



In [38]:
# # ======================= 📥 SELECIONAR COLUNAS =======================
# colunas = [
#     "id_prospect", 
#     "titulo", 
#     "nome_candidato", 
#     "codigo_candidato", 
#     "situacao_candidado",
#     "infos_basicas__objetivo_profissional",
#     "informacoes_profissionais__area_atuacao",
#     "formacao_e_idiomas__nivel_academico",
#     "formacao_e_idiomas__nivel_ingles",
#     "formacao_e_idiomas__nivel_espanhol",
#     "cv_pt", 
#     "perfil_vaga__competencia_tecnicas_e_comportamentais",
#     "perfil_vaga__nivel profissional",
#     "perfil_vaga__nivel_academico", 
#     "regex_termos_vaga", 
#     "termos_encontrados", 
#     "possui_termo", 
#     "qtd_termos"
# ]

# df = df[colunas].fillna('')

# # ======================= 🏷️ TARGET =======================
# status_positivo = [
#     'Encaminhado ao Requisitante',
#     'Contratado pela Decision',
#     'Contratado como Hunting',
#     'Aprovado',
#     'Entrevista Técnica',
#     'Entrevista com Cliente',
#     'Encaminhar Proposta',
#     'Proposta Aceita'
# ]

# status_negativo = [
#     'Não Aprovado pelo RH',
#     'Não Aprovado pelo Cliente',
#     'Não Aprovado pelo Requisitante',
#     'Desistiu',
#     'Desistiu da Contratação',
#     'Sem interesse nesta vaga',
#     'Recusado'
# ]

# df = df[df['situacao_candidado'].isin(status_positivo + status_negativo)]

# df['target'] = df['situacao_candidado'].apply(
#     lambda x: 1 if x in status_positivo else 0
# )

# # ======================= 🧹 FILTRO DE QUALIDADE =======================
# df['termos_encontrados'] = df['termos_encontrados'].apply(
#     lambda x: eval(x) if isinstance(x, str) else x
# )

# df = df[~((df['target'] == 1) & (df['termos_encontrados'].apply(lambda x: len(x) == 0)))]

# # ======================= 🔠 SIMILARIDADE TF-IDF =======================
# vectorizer = TfidfVectorizer()

# corpus = df['perfil_vaga__competencia_tecnicas_e_comportamentais'].astype(str).tolist() + \
#          df['cv_pt'].astype(str).tolist()

# tfidf_matrix = vectorizer.fit_transform(corpus)

# tfidf_vaga = tfidf_matrix[:len(df)]
# tfidf_cv = tfidf_matrix[len(df):]

# df['similaridade_cv_vaga'] = [
#     cosine_similarity(tfidf_cv[i], tfidf_vaga[i])[0][0] for i in range(len(df))
# ]

# # ======================= 📊 DENSIDADE DE TERMOS =======================
# df['densidade_termos'] = df['qtd_termos'] / (df['cv_pt'].str.split().str.len() + 1)

# # ======================= 🎓 FEATURE - ADERÊNCIA ACADÊMICA =======================
# def mapear_nivel_academico(nivel):
#     if pd.isna(nivel):
#         return 0
#     nivel = nivel.lower()
#     if "doutorado" in nivel:
#         return 5
#     if "mestrado" in nivel:
#         return 4
#     if "pós graduação" in nivel or "pós-graduação" in nivel:
#         return 3
#     if "ensino superior" in nivel:
#         return 2
#     if "ensino técnico" in nivel:
#         return 1.5
#     if "ensino médio" in nivel:
#         return 1
#     if "ensino fundamental" in nivel:
#         return 0.5
#     return 0

# df['nivel_academico_candidato'] = df['formacao_e_idiomas__nivel_academico'].apply(mapear_nivel_academico)
# df['nivel_academico_vaga'] = df['perfil_vaga__nivel_academico'].apply(mapear_nivel_academico)

# df['aderencia_academica'] = df.apply(
#     lambda x: 1 if x['nivel_academico_candidato'] >= x['nivel_academico_vaga'] else 0, axis=1
# )

# # ======================= 🌐 FEATURE - IDIOMAS =======================
# def mapear_nivel_idioma(nivel):
#     if pd.isna(nivel):
#         return 0
#     nivel = nivel.lower()
#     if nivel == 'nenhum':
#         return 0
#     if nivel == 'básico':
#         return 1
#     if nivel == 'intermediário':
#         return 2
#     if nivel == 'avançado':
#         return 3
#     if nivel == 'fluente':
#         return 4
#     return 0

# df['nivel_ingles'] = df['formacao_e_idiomas__nivel_ingles'].apply(mapear_nivel_idioma)
# df['nivel_espanhol'] = df['formacao_e_idiomas__nivel_espanhol'].apply(mapear_nivel_idioma)
# df['media_idiomas'] = (df['nivel_ingles'] + df['nivel_espanhol']) / 2

# # ======================= 💼 FEATURE - NÍVEL PROFISSIONAL DA VAGA =======================
# mapa_nivel_profissional = {
#     'Aprendiz': 1,
#     'Trainee': 2,
#     'Auxiliar': 3,
#     'Assistente': 4,
#     'Técnico de Nível Médio': 5,
#     'Júnior': 5.5,
#     'Analista': 6,
#     'Pleno': 7,
#     'Supervisor': 7,
#     'Líder': 7.5,
#     'Sênior': 8,
#     'Especialista': 9,
#     'Coordenador': 9,
#     'Gerente': 10
# }

# df['nivel_profissional_vaga'] = df['perfil_vaga__nivel profissional'].map(mapa_nivel_profissional).fillna(0)

# # ======================= 🎯 FEATURES FINAIS =======================
# features = [
#     'qtd_termos',
#     'similaridade_cv_vaga',
#     'densidade_termos',
#     'aderencia_academica',
#     'nivel_ingles',
#     'nivel_espanhol',
#     'nivel_profissional_vaga'
# ]

# X = df[features]
# y = df['target']

# # ======================= 🔧 NORMALIZAÇÃO =======================
# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)

# # ======================= ⚖️ BALANCEAMENTO =======================
# smote = SMOTE(random_state=42)
# X_res, y_res = smote.fit_resample(X_scaled, y)

# # ======================= 🚀 SPLIT =======================
# X_train, X_test, y_train, y_test = train_test_split(
#     X_res, y_res, test_size=0.2, random_state=42
# )

# # ======================= 🤖 MODELOS =======================
# modelos = {
#     'RandomForest': RandomForestClassifier(random_state=42),
#     'XGBoost': XGBClassifier(random_state=42, verbosity=0),
#     'LightGBM': LGBMClassifier(random_state=42),
#     'LogisticRegression': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
# }

# for nome, modelo in modelos.items():
#     modelo.fit(X_train, y_train)
#     y_pred = modelo.predict(X_test)
#     print(f"\n🔍 Modelo: {nome}")
#     print(classification_report(y_test, y_pred))

# # ======================= ✅ MODELO FINAL =======================
# modelo_final = RandomForestClassifier(random_state=42)
# modelo_final.fit(X_res, y_res)

# # ======================= 🔥 NOVO SCORE FINAL =======================
# df['prob_apto'] = modelo_final.predict_proba(X_scaled)[:, 1]

# # Normalização manual de variáveis complementares
# df['qtd_termos_norm'] = df['qtd_termos'] / df['qtd_termos'].max()
# df['densidade_termos_norm'] = df['densidade_termos'] / df['densidade_termos'].max()
# df['nivel_profissional_norm'] = df['nivel_profissional_vaga'] / 10
# df['media_idiomas_norm'] = df['media_idiomas'] / 4

# # Score mais balanceado e robusto
# df['score_final'] = (
#     (df['prob_apto'] * 0.5) +  
#     (df['similaridade_cv_vaga'] * 0.2) +  
#     (df['aderencia_academica'] * 0.1) +  
#     (df['media_idiomas_norm'] * 0.05) +  
#     (df['nivel_profissional_norm'] * 0.05) +  
#     (df['qtd_termos_norm'] * 0.05) +  
#     (df['densidade_termos_norm'] * 0.05)  
# )

# # ======================= 🏆 TOP CANDIDATOS =======================
# top_candidatos = df.sort_values('score_final', ascending=False)

In [39]:
# ======================= 🔥 CLASSIFICAÇÃO COM 2 THRESHOLDS =======================

# 🔥 Definir os thresholds
threshold_triagem = 0.4   # Para triagem inicial
threshold_shortlist = 0.5  # Para shortlist e encaminhamento final

# 📊 Classificar os candidatos em faixas
def classificar_candidato(prob):
    if prob >= threshold_shortlist:
        return '✅ Apto - Shortlist'
    elif prob >= threshold_triagem:
        return '🟨 Potencial - Triagem'
    else:
        return '❌ Não Apto'

df['status_candidato'] = df['prob_apto'].apply(classificar_candidato)

# ======================= 🏆 GERAR RANKING FINAL =======================

# 📈 Organizar ranking
df_rank = df.sort_values('score_final', ascending=False)

# ======================= 📊 VISUALIZAR QUANTITATIVO =======================

print("\n📊 Distribuição dos candidatos por status:\n")
print(df_rank['status_candidato'].value_counts())

# ======================= 🔍 FILTROS SEPARADOS =======================

# 🏆 Shortlist
shortlist = df_rank[df_rank['status_candidato'] == '✅ Apto - Shortlist']

# 🟨 Triagem
triagem = df_rank[df_rank['status_candidato'] == '🟨 Potencial - Triagem']

# ❌ Não Apto
nao_apto = df_rank[df_rank['status_candidato'] == '❌ Não Apto']


📊 Distribuição dos candidatos por status:

status_candidato
✅ Apto - Shortlist       19046
❌ Não Apto                2675
🟨 Potencial - Triagem     1360
Name: count, dtype: int64


In [40]:
# import joblib

# # Salvar o modelo treinado
# joblib.dump(modelo_final, 'modelo_rf.pkl')

# # Salvar o scaler que você usou
# joblib.dump(scaler, 'scaler.pkl')